# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.2475, -0.1429, -0.0264, -0.0509, -0.0417,  0.0437,  0.1688,  0.0966,
          0.2220, -0.2028],
        [-0.0042, -0.0497,  0.1269, -0.1065, -0.0126, -0.1436,  0.2482,  0.1191,
          0.1644, -0.1919]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-0.3603,  0.0978, -0.0143,  0.0487,  0.1333,  0.1802, -0.2279, -0.0343,
          0.1580, -0.0398],
        [-0.4157,  0.2255, -0.1704,  0.1343,  0.2084,  0.2736, -0.0576, -0.0667,
          0.1355, -0.1659]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-4.7026e-02, -6.7122e-02, -1.4189e-01, -2.7275e-03, -2.0876e-01,
         -3.0940e-01, -5.3935e-02,  3.9429e-02,  1.4583e-01, -1.2619e-01],
        [-1.9763e-02, -2.2275e-02, -7.0404e-03, -1.5859e-01, -1.0713e-01,
         -2.4011e-01, -2.0802e-01, -2.1527e-02,  2.3857e-01,  9.0867e-05]],
       grad_fn=<AddmmBackward>)

在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2215, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1322, grad_fn=<SumBackward0>)